In [1]:
from pathlib import Path
import subprocess

REPO_URL = "https://github.com/seoultechpse/fenicsx-colab.git"
ROOT = Path("/content")
REPO_DIR = ROOT / "fenicsx-colab"

subprocess.run(
  ["git", "clone", REPO_URL, str(REPO_DIR)],
  check=True
)

USE_CLEAN = False  # <--- Set True to remove existing environment
opts = "--clean" if USE_CLEAN else ""

get_ipython().run_line_magic(
    "run", f"{REPO_DIR / 'setup_fenicsx.py'} {opts}"
)

⚠️ Google Drive not mounted — using local cache (/content)
🔧 Installing FEniCSx environment...
✨ Loading FEniCSx Jupyter magic... %%fenicsx registered


In [2]:
%%fenicsx --info


🐍 Python          : 3.11.14
📦 dolfinx         : 0.10.0
💻 Platform        : Linux-6.6.105+-x86_64-with-glibc2.35
🧵 Running as root : True

🔎 fenicsx runtime info
-----------------------
Environment        : fenicsx
micromamba         : /content/micromamba/bin/micromamba
MPI implementation : OPENMPI
MPI version        : mpiexec (OpenRTE) 4.1.2
MPI ranks (-np)    : 1


---

In [3]:
%%fenicsx -np 4

import numpy as np
from mpi4py import MPI

import dolfinx.fem.petsc
from dolfinx import fem, io, mesh

from ufl import (
    SpatialCoordinate,
    TestFunction, TrialFunction,
    cos, dx, grad, inner
)

# Create mesh and define function space
msh = mesh.create_unit_cube(MPI.COMM_WORLD, 12, 16, 12)
V = fem . functionspace (msh, ("Lagrange", 3))

# Define variational problem
u, v = TrialFunction(V), TestFunction(V)
x, y, z = SpatialCoordinate(msh)
k = 4 * np.pi
f = (1.0 + 1.0j) * k**2 * cos(k * x) * cos(k * y)
a = inner(grad(u), grad(v)) * dx - k**2 * inner(u, v) * dx
L = inner(f, v) * dx

# Solve a(u, v) = L(v)
problem = dolfinx.fem.petsc.LinearProblem(
    a,
    L,
    petsc_options_prefix="helmholtz_",
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "ksp_error_if_not_converged": True},
)
uh = problem.solve()

# Save solution in VTX (.bp) format
with io.VTXWriter(msh.comm, "helmholtz.bp", [uh]) as vtx:
    vtx.write(0.0)

In [5]:
!zip -r helmholtz.zip helmholtz.bp

from google.colab import files
files.download('helmholtz.zip')

  adding: helmholtz.bp/ (stored 0%)
  adding: helmholtz.bp/data.0 (deflated 66%)
  adding: helmholtz.bp/md.idx (deflated 61%)
  adding: helmholtz.bp/mmd.0 (deflated 64%)
  adding: helmholtz.bp/md.0 (deflated 90%)
  adding: helmholtz.bp/profiling.json (deflated 80%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>